In [1]:
import numpy as np
import math
import random
import pandas as pd
import plotly.express as px
import multiprocessing


def sigmoid(z: np.array):
    return 1 / (1 + np.exp(-np.clip(z, -100, 100)))


def tanh(z: np.array):
    z = np.clip(z, -100, 100)
    expZ = np.exp(z)
    expZ_minus = np.exp(-z)
    return (expZ - expZ_minus) / (expZ + expZ_minus)


def reLU(z: np.array):
    return np.maximum(0.0, z)


def leakyReLU(z: np.array):
    return np.maximum(0.01 * z, z)

In [10]:
class Model:
    class Layer:
        def __init__(self, layerHeight, prevLayer, nextLayer, activationFunc="sigmoid"):
            self.layerHeight = layerHeight
            self.w = np.random.randn(layerHeight, prevLayer.layerHeight) * np.sqrt(1 / prevLayer.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.v_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.v_db = np.zeros((layerHeight, 1))
            self.s_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.s_db = np.zeros((layerHeight, 1))
            self.t = 1
            self.a = None
            self.d = None
            if activationFunc == "sigmoid":
                self.activation = sigmoid
                self.da_dz = lambda a: np.multiply(a, (1 - a))
            elif activationFunc == "tanh":
                self.activation = np.tanh
                self.da_dz = lambda a: (1 - np.square(a))
            elif activationFunc == "relu":
                self.activation = reLU
                self.da_dz = lambda a: np.where(a > 0, 1,
                                                0)  #just taking advantage of the fact a = z for positive and a = 0 for negative
            elif activationFunc == "leaky_relu":
                self.activation = leakyReLU
                self.da_dz = lambda a: np.where(a > 0, 1, 0.01)
            self.prevLayer = prevLayer if prevLayer else None
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.d = np.random.random((self.layerHeight, a_Prev.shape[1])) < keepActiveProb
            self.a = self.activation((self.w @ a_Prev) + self.b) * self.d
            self.a /= keepActiveProb
            # self.a = self.activation((self.w @ a_Prev) + self.b)
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, dz_Next: np.array, trainingStep: float, l2_lambda=0):
            da = (self.nextLayer.w.T @ dz_Next) * self.d
            dz = self.da_dz(self.a) * da
            dw = ((dz @ self.prevLayer.a.T) / dz.shape[1]) + ((l2_lambda / dz.shape[1]) * self.w)
            db = np.mean(dz, axis=1, keepdims=True)

            beta1 = 0.9
            beta2 = 0.999
            self.v_dw = (beta1 * self.v_dw) + ((1 - beta1) * dw)
            self.v_db = (beta1 * self.v_db) + ((1 - beta1) * db)
            self.s_dw = (beta2 * self.s_dw) + ((1 - beta2) * np.square(dw))
            self.s_db = (beta2 * self.s_db) + ((1 - beta2) * np.square(db))
            v_dw_cor = self.v_dw / (1 - (beta1 ** self.t))
            v_db_cor = self.v_db / (1 - (beta1 ** self.t))
            s_dw_cor = self.s_dw / (1 - (beta2 ** self.t))
            s_db_cor = self.s_db / (1 - (beta2 ** self.t))

            self.prevLayer.propagate(dz, trainingStep, l2_lambda)
            self.w -= trainingStep * (v_dw_cor / (np.sqrt(s_dw_cor) + (10 ** -8)))
            self.b -= trainingStep * (v_db_cor / (np.sqrt(s_db_cor) + (10 ** -8)))
            self.t += 1

    class Head(Layer):
        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.a = sigmoid((self.w @ a_Prev) + self.b)
            return self.a

        def propagate(self, realY, trainingStep, l2_lambda=0):
            dz = self.a - realY
            dw = ((dz @ self.prevLayer.a.T) / dz.shape[1]) + ((l2_lambda / dz.shape[1]) * self.w)
            db = np.mean(dz, axis=1, keepdims=True)

            beta1 = 0.9
            beta2 = 0.999
            self.v_dw = (beta1 * self.v_dw) + ((1 - beta1) * dw)
            self.v_db = (beta1 * self.v_db) + ((1 - beta1) * db)
            self.s_dw = (beta2 * self.s_dw) + ((1 - beta2) * np.square(dw))
            self.s_db = (beta2 * self.s_db) + ((1 - beta2) * np.square(db))
            v_dw_cor = self.v_dw / (1 - (beta1 ** self.t))
            v_db_cor = self.v_db / (1 - (beta1 ** self.t))
            s_dw_cor = self.s_dw / (1 - (beta2 ** self.t))
            s_db_cor = self.s_db / (1 - (beta2 ** self.t))

            self.prevLayer.propagate(dz, trainingStep)
            self.w -= trainingStep * (v_dw_cor / (np.sqrt(s_dw_cor) + (10 ** -8)))
            self.b -= trainingStep * (v_db_cor / (np.sqrt(s_db_cor) + (10 ** -8)))
            self.t += 1

    class Input(Layer):
        def __init__(self, layerHeight, nextLayer):
            self.layerHeight = layerHeight
            self.a = None
            self.w = np.identity(self.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.a = (self.w @ a_Prev) + self.b
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, realY, trainingStep, l2_lambda=0):
            return

    def __init__(self, xSize, layers, activationFunc="sigmoid"):
        """activationFunc = "sigmoid" | "tanh" | "relu" | "leaky_relu" """
        self.head = None

        def createLayer(prevLayer, depth):
            if depth >= len(layers):
                layer = self.Head(1, prevLayer, None)
                self.head = layer
                return layer
            else:
                layer = self.Layer(layers[depth], prevLayer, None, activationFunc=activationFunc)
                layer.nextLayer = createLayer(layer, depth + 1)
                return layer

        self.input = self.Input(xSize, None)
        self.input.nextLayer = createLayer(self.input, 0)
        self.trainingData = pd.DataFrame([], columns=['Name', 'Iteration', 'Value'])

    def predict(self, a: np.array, keepActiveProb=1.0):
        return self.input.predict(a, keepActiveProb)

    def train(self, X: np.array, Y: np.array, trainingStep, trainingIterations,
              debugStep=100, x_dev=None, y_dev=None, epochSize=1024, keepActiveProb=1.0, l2_reg_lambda=0):
        """
        epochSize ~ 128 | 256 | 512 \n
        keepActiveProb ~ ratio of active neurons {>0.8} \n
        l2_reg_lambda ~ weight decay coefficient {<2.0} \n
        """
        trainSetMean = np.mean(X, axis=1, keepdims=True)
        trainSetR2 = np.mean(np.square(X - trainSetMean), axis=1, keepdims=True)
        self.input.w = self.input.w / np.sqrt(trainSetR2)
        self.input.b = -trainSetMean / np.sqrt(trainSetR2)
        epochs = X.shape[1] // epochSize
        display('epochs Amount: ' + str(epochs))
        for tr_i in range(trainingIterations):
            perm_indices = np.random.permutation(X.shape[1])
            x = X[:, perm_indices]
            y = Y[:, perm_indices]
            for e in range(epochs):
                self.predict(x[:, epochSize * e: epochSize * (e + 1)], keepActiveProb)
                self.head.propagate(y[:, epochSize * e: epochSize * (e + 1)], trainingStep, l2_reg_lambda)
            if x_dev is not None and tr_i % debugStep == 0:
                self.predict(x_dev)
                self.trainingData.loc[len(self.trainingData)] = ('ErrorDev', tr_i // debugStep, -np.mean(
                    (y_dev * np.log(np.clip(self.head.a, 0.001, 1))) + (
                            (1 - y_dev) * np.log(np.clip(1 - self.head.a, 0.001, 1)))))
                self.trainingData.loc[len(self.trainingData)] = (
                    'PrecisionDev', tr_i // debugStep, np.mean((y_dev == (self.head.a > 0.5)).astype(int)))
            if tr_i % debugStep == 0:
                self.predict(x)
                self.trainingData.loc[len(self.trainingData)] = ('Error', tr_i // debugStep, -np.mean(
                    (y * np.log(np.clip(self.head.a, 0.001, 1))) + (
                            (1 - y) * np.log(np.clip(1 - self.head.a, 0.001, 1)))))
                self.trainingData.loc[len(self.trainingData)] = (
                    'Precision', tr_i // debugStep, np.mean((y == (self.head.a > 0.5)).astype(int)))


In [3]:
class Model_Concise:
    class Layer:
        def __init__(self, layerHeight, prevLayer, nextLayer, activationFunc="tanh"):
            self.layerHeight = layerHeight
            self.w = np.random.randn(layerHeight, prevLayer.layerHeight) * np.sqrt(1 / prevLayer.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.v_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.v_db = np.zeros((layerHeight, 1))
            self.s_dw = np.zeros((layerHeight, prevLayer.layerHeight))
            self.s_db = np.zeros((layerHeight, 1))
            self.t = 1
            self.a, self.z = None, None
            self.d = 1

            # self.dy_dz = lambda nextL, dz_Next, a, z: (nextL.w.T @ dz_Next)
            if activationFunc == "sigmoid":
                self.activation = sigmoid
                # self.da_dz = lambda a, z: np.multiply(a, (1 - a))
                self.dy_dz = lambda nextL, dz_Next, a, z: (nextL.w.T @ dz_Next) * (a - np.square(a))
            elif activationFunc == "tanh":
                self.activation = np.tanh
                # self.da_dz = lambda a, z: (1 - np.square(a))
                self.dy_dz = lambda nextL, dz_Next, a, z: (nextL.w.T @ dz_Next) * (1 - np.square(a))
            elif activationFunc == "relu":
                self.activation = reLU
                # self.da_dz = lambda a, z: np.where(z > 0, 1, 0)
                self.dy_dz = lambda nextL, dz_Next, a, z: (nextL.w.T @ dz_Next) * (np.where(z > 0, 1, 0))
            elif activationFunc == "leaky_relu":
                self.activation = leakyReLU
                # self.da_dz = lambda a, z: np.where(z > 0, 1, 0.01)
                self.dy_dz = lambda nextL, dz_Next, a, z: (nextL.w.T @ dz_Next) * (np.where(z > 0, 1, 0.01))
            elif activationFunc == "softmax":
                self.activation = softMax
                # self.da_dz = lambda a, z: a - np.square(a)
                self.dy_dz = lambda nextL, dz_Next, a, z: ValueError("softmax shouldn't be used for hidden layers")
            self.prevLayer = prevLayer if prevLayer else None
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.d = np.random.random((self.layerHeight, a_Prev.shape[1])) < keepActiveProb
            self.z = (self.w @ a_Prev) + self.b
            self.a = self.activation(self.z) * self.d
            self.a /= keepActiveProb
            # self.a = self.activation((self.w @ a_Prev) + self.b)
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, dz_Next: np.array, trainingStep: float, l2_lambda=0):
            # da = self.dy_da(self.a, dz_Next, self.nextLayer) * self.d
            # dz = self.da_dz(self.a, self.z) @ da
            dz = self.dy_dz(self.nextLayer, dz_Next, self.a, self.z) * self.d
            dw = ((dz @ self.prevLayer.a.T) / dz.shape[1]) + ((l2_lambda / dz.shape[1]) * self.w)
            db = np.mean(dz, axis=1, keepdims=True)

            beta1 = 0.9
            beta2 = 0.999
            self.v_dw = (beta1 * self.v_dw) + ((1 - beta1) * dw)
            self.v_db = (beta1 * self.v_db) + ((1 - beta1) * db)
            self.s_dw = (beta2 * self.s_dw) + ((1 - beta2) * np.square(dw))
            self.s_db = (beta2 * self.s_db) + ((1 - beta2) * np.square(db))
            v_dw_cor = self.v_dw / (1 - (beta1 ** self.t))
            v_db_cor = self.v_db / (1 - (beta1 ** self.t))
            s_dw_cor = self.s_dw / (1 - (beta2 ** self.t))
            s_db_cor = self.s_db / (1 - (beta2 ** self.t))

            self.prevLayer.propagate(dz, trainingStep, l2_lambda)
            self.w -= trainingStep * (v_dw_cor / (np.sqrt(s_dw_cor) + (10 ** -8)))
            self.b -= trainingStep * (v_db_cor / (np.sqrt(s_db_cor) + (10 ** -8)))
            self.t += 1

    class Head(Layer):
        def __init__(self, layerHeight, prevLayer, nextLayer, activationFunc="sigmoid-logistic"):
            super().__init__(layerHeight, prevLayer, nextLayer)
            if activationFunc == "sigmoid-logistic":
                self.activation = sigmoid
                self.dy_dz = lambda nextL, y, a, z: a - y
                self.error = lambda y_hat, y: np.sum(-np.mean(
                    (y * np.log(np.clip(y_hat, 0.001, 1))) + (
                            (1 - y) * np.log(np.clip(1 - y_hat, 0.001, 1))), axis=1, keepdims=True))
            elif activationFunc == "sigmoid-r2":
                self.activation = sigmoid
                self.dy_dz = lambda nextL, y, a, z: 2 * (a - y) * (a - np.square(a))
                self.error = lambda y_hat, y: np.mean(np.square(y - y_hat))
            elif activationFunc == "relu-r2":
                self.activation = reLU
                self.dy_dz = lambda nextL, y, a, z: 2 * (a - y) * (np.where(z > 0, 1, 0))
                self.error = lambda y_hat, y: np.mean(np.square(y - y_hat))
            elif activationFunc == "softmax-logistic":
                self.activation = softMax
                self.dy_dz = lambda nextL, y, a, z: a - y
                self.error = lambda y_hat, y: np.sum(-np.mean(
                    y * np.log(np.clip(y_hat, 0.001, 1)), axis=1, keepdims=True))

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.z = (self.w @ a_Prev) + self.b
            self.a = self.activation(self.z)
            return self.a

    class Input(Layer):
        def __init__(self, layerHeight, nextLayer):
            self.layerHeight = layerHeight
            self.a = None
            self.w = np.identity(self.layerHeight)
            self.b = np.zeros((layerHeight, 1))
            self.nextLayer = nextLayer if nextLayer else None

        def predict(self, a_Prev: np.array, keepActiveProb: float):
            self.a = (self.w @ a_Prev) + self.b
            return self.nextLayer.predict(self.a, keepActiveProb)

        def propagate(self, realY, trainingStep, l2_lambda=0):
            return

    def __init__(self, xSize, ySize, layers, activationFunc="tanh", headActivation="sigmoid-logistic"):
        """
        activationFunc = "sigmoid" | "tanh" | "relu" | "leaky_relu" | "softmax" \n
        headActivation = "sigmoid-logistic" | "sigmoid-r2" | "relu-r2" | "softmax-logistic"
        """
        self.head = None

        def createLayer(prevLayer, depth):
            if depth >= len(layers):
                layer = self.Head(ySize, prevLayer, None, activationFunc=headActivation)
                self.head = layer
                return layer
            else:
                layer = self.Layer(layers[depth], prevLayer, None, activationFunc=activationFunc)
                layer.nextLayer = createLayer(layer, depth + 1)
                return layer

        self.input = self.Input(xSize, None)
        self.input.nextLayer = createLayer(self.input, 0)
        self.trainingData = pd.DataFrame([], columns=['Name', 'Iteration', 'Value'])

    def predict(self, a: np.array, keepActiveProb=1.0):
        return self.input.predict(a, keepActiveProb)

    def train(self, X: np.array, Y: np.array, trainingStep, trainingIterations,
              debugStep=100, x_dev=None, y_dev=None, epochSize=1024, keepActiveProb=1.0, l2_reg_lambda=0):
        """
        epochSize ~ 128 | 256 | 512 \n
        keepActiveProb ~ ratio of active neurons {>0.8} \n
        l2_reg_lambda ~ weight decay coefficient {<2.0} \n
        """
        trainSetMean = np.mean(X, axis=1, keepdims=True)
        trainSetR2 = np.mean(np.square(X - trainSetMean), axis=1, keepdims=True)
        self.input.w = self.input.w / np.sqrt(trainSetR2)
        self.input.b = -trainSetMean / np.sqrt(trainSetR2)
        epochs = X.shape[1] // epochSize
        display('epochs Amount: ' + str(epochs))
        for tr_i in range(trainingIterations):
            perm_indices = np.random.permutation(X.shape[1])
            x = X[:, perm_indices]
            y = Y[:, perm_indices]
            for e in range(epochs):
                self.predict(x[:, epochSize * e: epochSize * (e + 1)], keepActiveProb)
                self.head.propagate(y[:, epochSize * e: epochSize * (e + 1)], trainingStep, l2_reg_lambda)
            if x_dev is not None and tr_i % debugStep == 0:
                self.predict(x_dev)
                self.trainingData.loc[len(self.trainingData)] = (
                    'ErrorDev', tr_i // debugStep, self.head.error(self.head.a, y_dev))
                self.trainingData.loc[len(self.trainingData)] = (
                    'PrecisionDev', tr_i // debugStep, np.mean((y_dev == (self.head.a > 0.5)).astype(int)))
            if tr_i % debugStep == 0:
                self.predict(x)
                self.trainingData.loc[len(self.trainingData)] = (
                    'Error', tr_i // debugStep, self.head.error(self.head.a, y))
                self.trainingData.loc[len(self.trainingData)] = (
                    'Precision', tr_i // debugStep, np.mean((y == (self.head.a > 0.5)).astype(int)))


In [4]:
#Prepare Training Set
trainCasesAmount = 100000
digitAmount = 10
x = np.zeros((digitAmount, trainCasesAmount), dtype=float)
y = np.zeros((1, trainCasesAmount), dtype=float)
for j in range(trainCasesAmount):
    if random.random() < 0.5:
        startI = min(int(-10 * (random.random() - 1)), 8)
        x[:startI, j] = [0]
        for i in range(startI, digitAmount):
            x[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
        ch = random.randint(startI, 9)
        x[ch, j] = x[-(ch - startI) - 1, j] + 1 if x[-(ch - startI) - 1, j] < 9 else x[-(ch - startI) - 1, j] - 1
        y[0, j] = 0
    else:
        startI = min(int(-10 * (random.random() - 1)), 9)
        for i in range(startI, startI + ((digitAmount - startI) // 2) + 1):
            x[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
            x[-(i - startI) - 1, j] = x[i, j]
        y[0, j] = 1

C:\Users\Admin\AppData\Local\Temp\ipykernel_9532\3711709577.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9532\3711709577.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)


In [5]:
#Prepare Dev Set
devCasesAmount = 1000
digitAmount = 10
x_dev = np.zeros((digitAmount, trainCasesAmount), dtype=float)
y_dev = np.zeros((1, trainCasesAmount), dtype=float)
for j in range(trainCasesAmount):
    if random.random() < 0.5:
        startI = min(int(-10 * (random.random() - 1)), 8)
        x_dev[:startI, j] = [0]
        for i in range(startI, digitAmount):
            x_dev[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
        ch = random.randint(startI, 9)
        x_dev[ch, j] = x_dev[-(ch - startI) - 1, j] + 1 if x_dev[-(ch - startI) - 1, j] < 9 else x_dev[-(
                    ch - startI) - 1, j] - 1
        y_dev[0, j] = 0
    else:
        startI = min(int(-10 * (random.random() - 1)), 9)
        for i in range(startI, startI + ((digitAmount - startI) // 2) + 1):
            x_dev[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
            x_dev[-(i - startI) - 1, j] = x_dev[i, j]
        y_dev[0, j] = 1

C:\Users\Admin\AppData\Local\Temp\ipykernel_9532\927766409.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_dev[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9532\927766409.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_dev[i, j] = np.random.randint(0 if i > startI else 1, 10, 1)


In [11]:
np.random.seed(69)
model0 = Model(10, [20, 20], activationFunc="tanh")
model0.train(x, y, 0.01, 101, x_dev=x_dev, y_dev=y_dev, epochSize=512, debugStep=10)

'epochs Amount: 195'

In [ ]:
np.random.seed(69)
model0_minus = Model_Old(10, [20, 20], activationFunc="tanh")
model0_minus.train(x, y, 1, 2001, x_dev=x_dev, y_dev=y_dev, debugStep=10)

In [7]:
np.random.seed(69)
model_small = Model_Concise(10, 1, [20, 20], activationFunc="tanh", headActivation="sigmoid-logistic")
model_small.train(x, y, 0.01, 101, x_dev=x_dev, y_dev=y_dev, epochSize=512, debugStep=10)

'epochs Amount: 195'

In [ ]:
np.random.seed(69)
model2 = Model(10, [10, 10], activationFunc="tanh")
model2.train(x, y, 0.1, 200000, 0.95, devSet=True, x_dev=x_dev, y_dev=y_dev)

In [12]:
px.line(model0.trainingData, x='Iteration', y='Value', color='Name')

In [9]:
px.line(model_small.trainingData, x='Iteration', y='Value', color='Name')

In [15]:
inp = input()
while inp != 'stop':
    if len(inp) < 10:
        inp = "0" * (10 - len(inp)) + inp
    print(inp + ' is ' + str(model_small.predict(np.array([[int(i)] for i in inp])).item()))
    inp = input()

0000012321 is 0.9893987788501528
0000124521 is 0.0006483604005645902
0000000101 is 0.9997238583796199
0000000111 is 0.9992393073075545
0000124421 is 0.8946809913613158
0087567865 is 1.214623255468863e-10
0000008787 is 5.651715697593757e-06
0000078492 is 4.324174597919076e-10
0000000857 is 0.0016221927456579798
0000008374 is 5.094397339301035e-06
0000000834 is 0.0069143051253774575
0000000100 is 1.3478796422934242e-08
0000000010 is 3.969659335571694e-05
0000000001 is 0.9999825450335169
0000000000 is 0.9999999984895358
0000898989 is 1.8985454591848867e-08
0000087877 is 0.0001382617492533818
0000003454 is 8.858117175261641e-05
0000003432 is 7.121530275932771e-05
0000003430 is 0.0003442869484233553
0000008345 is 1.639907423773536e-06


In [10]:
model0.predict(x)

array([[9.96504117e-01, 9.99999998e-01, 2.17117070e-08, ...,
        2.31224128e-04, 1.69326022e-08, 9.99970970e-01]])